In [1]:
import os
import sys
import base64
import argparse
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score
from tqdm import tqdm
import wandb
from utils import read_image, create_wandb_run_name, calculate_subgroup_metrics
sys.path.append("/mnt/data2/datasets_lfay/MedImageInsights")
from MedImageInsight.medimageinsightmodel_with_lora import MedImageInsight

/mnt/data2/datasets_lfay/MedImageInsights/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = MedImageInsight(
    model_dir="/mnt/data2/datasets_lfay/MedImageInsights/MedImageInsight/2024.09.27",
    vision_model_name="medimageinsigt-v1.0.0.pt",
    language_model_name="language_model.pth"
)

lora_config = {
    "r": 8,
    "lora_alpha": 16,
    "target_modules": [
        "image_encoder.blocks.0.0.spatial_block.window_attn.fn.qkv",
        "image_encoder.blocks.0.0.spatial_block.window_attn.fn.proj",
        "image_encoder.blocks.1.0.channel_block.channel_attn.fn.qkv",
        "image_encoder.blocks.1.0.channel_block.channel_attn.fn.proj"
    ],
    "lora_dropout": 0.1,
    "bias": "none",
    "task_type": "SEQ_CLS"  # Change task type if needed
}

classifier.load_model(use_lora=True, lora_config=lora_config)


Model loaded successfully on device: cuda


In [3]:
PATH_TO_DATA = "/mnt/data2/datasets_lfay/MedImageInsights/data"
data_path = os.path.join(PATH_TO_DATA, "MIMIC-v1.0-512")

df_train = pd.read_csv(os.path.join(data_path, "train.csv"))
df_val = pd.read_csv(os.path.join(data_path, "val.csv"))
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))

In [4]:
# Balance datasets
def balance_dataset(df, disease_column):
    positive_count = df[disease_column].sum()

    if args.only_no_finding:
        balanced_df = pd.concat([
            df[df[disease_column] == 1].sample(n=int(positive_count), random_state=42),
            df[df["No Finding"] == 1].sample(n=int(positive_count), random_state=42)
        ])
    else:
        balanced_df = pd.concat([
            df[df[disease_column] == 0].sample(n=int(positive_count), random_state=42),
            df[df[disease_column] == 1]
        ])
    return balanced_df

In [5]:
PATH_TO_DATA = "/mnt/data2/datasets_lfay/MedImageInsights/data"
data_path = os.path.join(PATH_TO_DATA, "MIMIC-v1.0-512")


df_train = pd.read_csv(os.path.join(data_path, "train.csv"))
df_val = pd.read_csv(os.path.join(data_path, "val.csv"))
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))

In [6]:
df_train = df_train[(df_train["No Finding"] == 1) | (df_train["Pneumonia"] == 1)]
df_val = df_val[(df_val["No Finding"] == 1) | (df_val["Pneumonia"] == 1)]
df_test = df_test[(df_test["No Finding"] == 1) | (df_test["Pneumonia"] == 1)]



In [ ]:
from dataset.PneumoniaDataset import PneumoniaDataset
from torch.utils.data import DataLoader

# Define base directory for images
data_dir = "/path/to/data"

# Create dataset
train_dataset = PneumoniaDataset(df=df_train, data_dir=PATH_TO_DATA)
val_dataset = PneumoniaDataset(df=df_val, data_dir=PATH_TO_DATA)
test_dataset = PneumoniaDataset(df=df_test, data_dir=PATH_TO_DATA)
# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
# Example usage
# for batch in train_loader:
#     images, labels = batch
    # print(f"Batch of base64 images: {images[:2]}")  # First two base64-encoded images
    # print(f"Batch of labels: {labels[:2]}")        # Corresponding labels


In [8]:
for batch in train_loader:
    images, labels = batch
    break

In [9]:
# Fine-tune the model
classifier.fine_tune(train_loader=train_loader, val_loader=val_loader, epochs=10, lr=5e-4)

# Save the fine-tuned LoRA parameters
#classifier.save_lora("./lora_pneumonia_finetuned")


AttributeError: 'UniCLModel' object has no attribute 'config'

NameError: name 'model_dir' is not defined